IMPORTING RELEVENT LIBRARIES

In [20]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

DATA

In [21]:
mnist_dataset , mnist_info =  tfds.load(name = 'mnist', with_info=True, as_supervised=True) #loading the data

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples,tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples,tf.int64)

def scale(image,lable):
  image = tf.cast(image,tf.float32)
  image /= 255.
  return image,lable

scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)
# SHUFFELING THE DATASET SO THAT MODEL WON'T GET CONGUSED DURING MINI-BATCH GD
BUFFER_SIZE = 10000 #when we are dealing with enormus datasets we can't suffle all data at once

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)

train_data = shuffled_train_and_validation_data.skip(num_validation_samples)
# CONVETING TRAIN DATASET TO BATCHES TO PERFORM MINI-BATCH GD
BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

# we want out validation set to be as same shape as out train and test data as (as_supervised = True)
validation_inputs,validation_targets = next(iter(validation_data))



OUTLINE THE MODEL

In [34]:
input_size = 784
output_size = 10
hidden_layer_size = 100

model = tf.keras.Sequential([
            
            tf.keras.layers.Flatten(input_shape=(28,28,1)),
            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
            tf.keras.layers.Dense(output_size, activation='softmax')
])

OPTIMIZER AND LOSS FUNCTION

In [36]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

TRAINING THE DATA

In [37]:
NUM_EPOCHS = 5  

model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose =2)


Epoch 1/5
540/540 - 4s - loss: 0.3328 - accuracy: 0.9054 - val_loss: 0.1691 - val_accuracy: 0.9493 - 4s/epoch - 8ms/step
Epoch 2/5
540/540 - 4s - loss: 0.1385 - accuracy: 0.9585 - val_loss: 0.0988 - val_accuracy: 0.9692 - 4s/epoch - 7ms/step
Epoch 3/5
540/540 - 4s - loss: 0.0952 - accuracy: 0.9713 - val_loss: 0.0825 - val_accuracy: 0.9752 - 4s/epoch - 8ms/step
Epoch 4/5
540/540 - 4s - loss: 0.0734 - accuracy: 0.9779 - val_loss: 0.0639 - val_accuracy: 0.9810 - 4s/epoch - 7ms/step
Epoch 5/5
540/540 - 4s - loss: 0.0598 - accuracy: 0.9820 - val_loss: 0.0581 - val_accuracy: 0.9820 - 4s/epoch - 7ms/step


TESTING THE MODEL

In [38]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 2s 2s/step - loss: 0.0853 - accuracy: 0.9728


In [42]:
print('Test loss: {0:.2f}.Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100))

Test loss: 0.09.Test accuracy: 97.28%
